<a href="https://colab.research.google.com/github/BayMichal/AI_Python/blob/main/Breast_cancer_PyTorch_v01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install torchviz

In [ ]:
pip install hiddenlayer

In [ ]:
!pip install -q gwpy

In [ ]:
# Import Libraries
import os
import numpy as np
import matplotlib.pyplot as plt
import tarfile
from tqdm.notebook import tqdm
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, classification_report
import seaborn as sns
import pandas as pd
import time
from torchviz import make_dot
import hiddenlayer as hl

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchvision.datasets.utils import download_url
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torchvision.transforms as tt
from torch.utils.data import random_split
from torchvision.utils import make_grid
from torchvision import transforms
import torchvision.models as models

project_name='Breast-Cancer'


In [ ]:
!mkdir ~/.kaggle #Tworze katalog kaggle w google collab 



mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
#Wrzucam plik do gog colab (usuwa sie po sesji)

! cp /kaggle.json ~/.kaggle/ # kopia pliku do folderu kaagle
#!echo '<"username":"volatille","key":"b06bccd3d1795d29ff0b071f635f2d79">' > ~/.kaggle/kaggle.json #write kaggle API credentials to kaggle.json
!chmod 600 ~/.kaggle/kaggle.json  #Ustawienie zgody na dostep

In [ ]:
!pip install -q kaggle #Install kaggle

In [ ]:
import kaggle

#Ściagam baze danych
!kaggle datasets download "paultimothymooney/breast-histopathology-images"


100% 3.10G/3.10G [00:48<00:00, 72.1MB/s]
100% 3.10G/3.10G [00:48<00:00, 68.9MB/s]


In [ ]:
#Rozpakowywuje baze danych, ukrywam output, bo za dużo śmieci
%%capture
!unzip breast-histopathology-images.zip

In [ ]:
#Wczesniej wczytywałem poprzez sciezkie w systemie, po migracji na coolab zmieniłem ścieżkę.

data_dir = '../content'
folder_name = "IDC_regular_ps50_idx5"
image_folders = os.path.join(data_dir, folder_name)

transform = transforms.Compose([transforms.Resize((50, 50)), transforms.ToTensor()]) #Funkcja robie resize każdego pliku (50x50) oraz rzutuję na = torch.FloatTensor of shape (C x H x W) in the range [0.0, 1.0] 

zdjecia_idc = []
for file in os.listdir(image_folders):
    zdjecia_idc.append(ImageFolder(os.path.join(image_folders, file), transform=transform)) #pętla, alokacja wczesniej przyjętych zmian dla plików.
datasets = torch.utils.data.ConcatDataset(zdjecia_idc) #upchanie wszystkiego do datasets

In [ ]:
#Sprawdzenie poprawnosci Datasets
datasets

In [ ]:
from collections import Counter

#Obliczanie ile jest danych w zbiorze (IDC)
j=0
for dataset in tqdm(datasets.datasets):
    if j == 0:
        result = Counter(dataset.targets)
        j += 1
    else:
        result += Counter(dataset.targets)


Pliki_2 = os.listdir(os.path.join(data_dir, folder_name))
print(" \n Liczba Pacjentów:", len(Pliki_2))

print("""\n Liczba zdjęć:
    IDC_0 (Brak IDC)  : {}
    IDC_i (Obecne IDC): {}""".format(result[0], result[1]))




  0%|          | 0/279 [00:00<?, ?it/s]

 
 Liczba Pacjentów: 279

 Liczba zdjęć:
    IDC_0 (Brak IDC)  : 198738
    IDC_i (Obecne IDC): 78786


In [ ]:
# Przygotowywanie modelu,danych itp.
random_seed = 42
torch.manual_seed(random_seed)

test_size = 38000
train_size = len(datasets) - test_size
train_ds, test_ds = random_split(datasets, [train_size, test_size])

val_size = 38000
train_size = len(train_ds) - val_size
train_ds, val_ds = random_split(train_ds, [train_size, val_size])

len(train_ds), len(val_ds), len(test_ds)

(201524, 38000, 38000)

In [ ]:
#Dane do treningu, validacji oraz testu
#   shuffle=True, przy false może nastąpić większy błąd val_loss (w treningu sieci)
#   pin_memory to wstawka dla conda cpu

train_data = DataLoader(train_ds, shuffle=True, num_workers=4, pin_memory=True) 
val_data = DataLoader(val_ds, shuffle=True, num_workers=4, pin_memory=True)
test_data = DataLoader(test_ds, shuffle=True, num_workers=4, pin_memory=True)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
# Transformacja danych /tt.ToTensor 
train_transform = tt.Compose([tt.ToTensor()])
valid_transform = tt.Compose([tt.ToTensor()])

In [ ]:
#     Batch_size - liczba próbek propagowanych przez uczącą się sieć. Czyli jesli równe 1000, to pobiera 1000 uczy sieć, potem znów pobiera 1000-2000 (1k) i uczy sieć.
#     problemem z batch size może być liczba próbek niepodzielna przez parametr ustawiony przez nas.

batch_size = 1000
# Data loadery. Podajemy dane dla treningu, validacji, testu.
#     Numworkers = 3, tutaj experymentalnie 3. 
#     Numworkers = 0 to znaczy że jest to główny proces
#     Numworkers = 1, oznacza że proces podzielony i ma jednego "podwykonawcę" (wedługp rzeczytanych przykładów, zazwyczaj tyle wstarczy do poprawnej implementacji)
#     Im więcej num_workers tym większe zasoby zuzywamy, a często nic nie zyskujemy. Możliwa obserwacja i wyciągnięcie wniosków poprzez wartości train_loss oraz val_loss

#     Shuffle, tasujemy dane treningowe.


train_dl = DataLoader(train_ds, batch_size, shuffle=True, num_workers=3, pin_memory=True)
val_dl = DataLoader(val_ds, batch_size*2, num_workers=3, pin_memory=True)
test_dl = DataLoader(test_ds, batch_size*2, num_workers=3, pin_memory=True)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 3 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
### Funkcja wskaźnika F1 według algorytmu ###
#   Funkcja jest testem na dokładność. Wartość 1.0 oznacza najwyższą precyzję

#Funkcja oblicza wartości:
# TP = True Positives
# TN = True Negatives
# FP = False Positives
# FN = False Negatives



def F_score(output, label, threshold=0.5, beta=1):
    prob = output > threshold
    label = label > threshold

    TP = (prob & label).sum(1).float()
    TN = ((~prob) & (~label)).sum(1).float()
    FP = (prob & (~label)).sum(1).float()
    FN = ((~prob) & label).sum(1).float()

    precision = torch.mean(TP / (TP + FP + 1e-12))
    recall = torch.mean(TP / (TP + FN + 1e-12))
    F2 = (1 + beta**2) * precision * recall / (beta**2 * precision + recall + 1e-12)
    return F2.mean(0)

In [ ]:
#Klasa zawierająca funkcję do kroku treningu modułu, kroku validacji modułu.
# @training_step() zwraca train_loss
# @validation_step() zwraca validation_loss
# @validation_epoch_end() 


class ImageClassificationBase(nn.Module):
    def training_step(self, batch):
        zdjecia_idc, targets = batch 
        targets = torch.reshape(targets.type(torch.cuda.FloatTensor), (len(targets), 1))
        out = self(zdjecia_idc)                      
        loss = F.binary_cross_entropy(out, targets)      
        return loss
    
    def validation_step(self, batch):
        zdjecia_idc, targets = batch
        targets = torch.reshape(targets.type(torch.cuda.FloatTensor), (len(targets), 1))
        out = self(zdjecia_idc)                      #Generowanie prognozy
        loss = F.binary_cross_entropy(out, targets)  #Oblicznaie strat
        score = F_score(out, targets)
        return {'val_loss': loss.detach(), 'val_score': score.detach() }
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Łączy straty
        batch_scores = [x['val_score'] for x in outputs]
        epoch_score = torch.stack(batch_scores).mean()  # Łączy dokładność
        return {'val_loss': epoch_loss.item(), 'val_score': epoch_score.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], last_lr: {:.4f}, train_loss: {:.4f}, val_loss: {:.4f}, val_score: {:.4f}".format(
            epoch, result['lrs'][-1], result['train_loss'], result['val_loss'], result['val_score']))

In [ ]:
class BreastCancerLogisticReg(ImageClassificationBase):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(input_size, 1)
    
    def forward(self, xb):
        xb = xb.reshape(-1, input_size)
        out = self.linear(xb)
        return torch.sigmoid(out)

In [ ]:
def get_default_device():
    """CUDA GPU lub jesli nie dostepne to CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    
def to_device(data, device):
    """Przenieś tensory do urzadzenia"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    """Dataloader/ przeniesienie danych do urządzenia"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Zarządzaj danymi po przeniesieniu"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Liczba partii"""
        return len(self.dl)

#Sprawdź czy GPU działa
device = get_default_device()
device

device(type='cuda')

In [ ]:
#Dataloader - moduł który zarządza danymi. Obsługuje potrzebne operacje na danych dozowanie, multi-processing, memory pinning.

#DATA TRAINING
train_dl = DeviceDataLoader(train_dl, device)

#DATA VALIDATION
val_dl = DeviceDataLoader(val_dl, device)

#DATA TESTING
test_dl = DeviceDataLoader(test_dl, device)

In [ ]:
def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

def fit_one_cycle(epochs, max_lr, model, train_loader, val_loader, 
                  weight_decay=0, grad_clip=None, opt_func=torch.optim.SGD):
    torch.cuda.empty_cache()
    history = []
    
    # Set up cutom optimizer with weight decay
    optimizer = opt_func(model.parameters(), max_lr, weight_decay=weight_decay)
    # Set up one-cycle learning rate scheduler
    sched = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr, epochs=epochs, 
                                                steps_per_epoch=len(train_loader))
    
    for epoch in range(epochs):
        # Training Phase 
        model.train()
        train_losses = []
        lrs = []
        for batch in tqdm(train_loader):
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            
            # Gradient clipping
            if grad_clip: 
                nn.utils.clip_grad_value_(model.parameters(), grad_clip)
            
            optimizer.step()
            optimizer.zero_grad()
            
            # Record & update learning rate
            lrs.append(get_lr(optimizer))
            sched.step()
        
        # Validation phase
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        result['lrs'] = lrs
        model.epoch_end(epoch, result)
        history.append(result)
    return history

In [ ]:
#Wielkośc wejścia sieci
input_size = 50*50*3

#Wybieramy dane do modelu sieci, dodajemy urzadzenie(GPU/CPU)
#  REGRESJA LOGISTYCZNA
model = to_device(BreastCancerLogisticReg(), device)

In [ ]:
# Przypisujemy zwracaną walidację przez funkcję evaluate naszego modelu 
history = [evaluate(model, val_dl)]

#Wyświetlamy
history

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 3 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


[{'val_loss': 0.7699790596961975, 'val_score': 0.28481578826904297}]

In [ ]:
#Liczba epok nauki sieci
epochs = 30

#Max szybkośc uczenia
max_lr = 0.01

#Algorytm optymalizujący = Adam
opt_func = torch.optim.Adam

In [ ]:
#Nauka sieci + parametry wizualizacji.

%%time
start_time = time.time()
history += fit_one_cycle(epochs, max_lr, model, train_dl, val_dl, opt_func=opt_func)

train_time = time.time() - start_time

  0%|          | 0/202 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 3 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch [0], last_lr: 0.0007, train_loss: 0.5231, val_loss: 0.4665, val_score: 0.0887


  0%|          | 0/202 [00:00<?, ?it/s]

Epoch [1], last_lr: 0.0015, train_loss: 0.4449, val_loss: 0.4495, val_score: 0.1160


...

Epoch [27], last_lr: 0.0002, train_loss: 0.4301, val_loss: 0.4402, val_score: 0.1511


  0%|          | 0/202 [00:00<?, ?it/s]

Epoch [28], last_lr: 0.0001, train_loss: 0.4297, val_loss: 0.4400, val_score: 0.1473


  0%|          | 0/202 [00:00<?, ?it/s]

Epoch [29], last_lr: 0.0000, train_loss: 0.4284, val_loss: 0.4399, val_score: 0.1463
CPU times: user 1min 58s, sys: 1min 15s, total: 3min 14s
Wall time: 53min 37s


In [ ]:
#Zablokuj kalkulację gradientu
@torch.no_grad()
def predict_dl(dl, model, threshold=0.5):
    torch.cuda.empty_cache()
    batch_probs = []
    for xb, _ in tqdm(dl):
        probs = model(xb)
        batch_probs.append(probs.cpu().detach())
    batch_probs = torch.cat(batch_probs)
    return [int(x) for x in batch_probs>threshold]

In [ ]:
#
def multi_acc(y_pred, y_test):
    y_pred_softmax = torch.log_softmax(y_pred, dim = 1)
    _, y_pred_tags = torch.max(y_pred_softmax, dim = 1)    
    correct_pred = (y_pred_tags == y_test).float()
    acc = correct_pred.sum() / len(correct_pred)
    acc = torch.round(acc * 100)
    return acc

In [ ]:
# 
IDC_PRED=[]
IDC_TRUE=[]
with torch.no_grad():
    with tqdm(test_dl, unit="batch") as tepoch:
        for inp, labels in tepoch:
            inp, labels = inp.to(device), labels.to(device)
            y_test_pred = model(inp)
            acc = multi_acc(y_test_pred, labels)
            _, y_pred_tag = torch.max(y_test_pred, dim = 1)
            tepoch.set_postfix(accuracy = acc.item())
            IDC_PRED.append(y_pred_tag.cpu().numpy())
            IDC_TRUE.append(labels.cpu().numpy())

NameError: ignored

In [ ]:
# flatten prediction and true lists
flat_pred = []
flat_true = []
for i in range(len(IDC_PRED)):
    for j in range(len(IDC_PRED[i])):
        flat_pred.append(IDC_PRED[i][j])
        flat_true.append(IDC_TRUE[i][j])
        
print(f"number of testing samples results: {len(flat_pred)}")

In [ ]:
#Testing accuracy. Wynik dokładności ai

print(f"Dokładność wynosi: {accuracy_score(flat_true, flat_pred) * 100:.2f}%")

In [ ]:
# Macierz błędu

print(classification_report(flat_true, flat_pred))